In \[12\]:

    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    from psycopg2.extras import execute_batch
    import psycopg2

In \[13\]:

    def extract_data(page):
        url = f'https://www.totaljobs.com/jobs/python/in-london?radius=30&page={page}'
        r = requests.get(url)
        soup = BeautifulSoup(r.content , 'html.parser')
        return soup

    joblist = []

In \[14\]:

    def transform(soup):      
        divs = soup.find_all('div', class_ = "Wrapper-sc-11673k2-0 eHVkAX")
        for item in divs:

            title = item.find('h2').text
            location = item.find('li', class_ = 'sc-fznXWL').text
            description = item.find('div', class_ = 'sc-fzoYkl').text.strip()
            salary = item.find('dl' , class_ = 'sc-fzoJMP').text
            date = item.find('li' , class_ = 'sc-fznXWL jwFffy').text
            company = item.find('div', class_ = 'sc-fzoiQi').text

            tech_list = ['Python', 'Cloud', 'Rust','TensorFlow', 'PyTorch', 'Keras', 'OpenCV' ,'Natural Language Toolkit', 'NLTK','scikit learn',
                  'Microsoft Cognitive Toolkit','Theano','Caffe', 'MXNet', 'H20.ai','IBM Watson Studio' , 'IBM Watson Assistant' ,'Google Cloud AutoML',
                  'Azure','Kubernetes', 'c' , 'c++' , 'Apache Spark', 'Pyspark', 'ECMAscript','TypeScript','Gitlab', 'SQL', 'NoSQL' ,  'AWS,Data Lake', 
                  'Redshift','Glue', 'Celery', 'RabbitMQ', 'Airflow' , 'LINUX' ,'Java','Machine Learning','Go']
            tech_list1 = [x.lower() for x in tech_list]
            technology = [string for string in tech_list1 if string in description.lower()]

            job ={
                  'Title' : title,
                  'Location' : location,
                  'Description' : description,
                  'Salary' : salary,
                  'Date_Posted' : date,
                  'Company' : company,
                  'Technology': technology
                 }

            joblist.append(job)
      
        return joblist

In \[16\]:

    for i in [1,2]:
        c = extract_data(i)
        job_list = transform(c)

In \[19\]:

    conn = psycopg2.connect(host='localhost',database='postgres', user='postgres',password='Kunal',port='5432')

    curs = conn.cursor()

    curs.execute('''CREATE TABLE IF NOT EXISTS jobs ( 
                    Title TEXT NOT NULL, 
                    Location TEXT NOT NULL, 
                    Description TEXT NOT NULL, 
                    Salary TEXT NOT NULL, 
                    Date_Posted TEXT NOT NULL, 
                    Company TEXT NOT NULL,
                    Technology TEXT NOT NULL)''')

    conn.commit()

    query = '''INSERT INTO jobs VALUES (%(Title)s,
                %(Location)s,
                %(Description)s,
                %(Salary)s,
                %(Date_Posted)s,
                %(Company)s,
                %(Technology)s)'''

In \[25\]:

    execute_batch (curs, query,job_list)

    conn.commit()
    curs.close()
    conn.close()